Dans ce code, on essaie de s'intéresser à pourquoi différents réseaux neuronaux classificateurs assignent la même classe à un exemple adversarial. L'hypothèse première est que c'est la linéarité de la méthode d'entraînement qui amènerait différents réseaux à classifier de la même manière les exemples adversariaux, en effet, les algorithmes de Machine Learning classifient assez bien pour généraliser.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression

# Charger les données MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train_flat = x_train.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)

# Fonction pour générer des exemples adversariaux (FGSM)
def generate_adversarial_examples(model, x, y, epsilon=0.25):
    adv_examples = []
    for i in range(len(x)):
        x_sample = tf.convert_to_tensor(x[i].reshape(1, 28, 28), dtype=tf.float32)
        y_sample = tf.convert_to_tensor([y[i]], dtype=tf.int64)
        with tf.GradientTape() as tape:
            tape.watch(x_sample)
            pred = model(x_sample)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_sample, pred)
        gradient = tape.gradient(loss, x_sample)
        perturbation = epsilon * tf.sign(gradient)
        x_adv = x_sample + perturbation
        x_adv = tf.clip_by_value(x_adv, 0, 1)
        adv_examples.append(x_adv.numpy().squeeze())
    return np.array(adv_examples)

# Modèle Maxout simulé avec Dense
def create_maxout_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Modèle Softmax peu profond
def create_softmax_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Modèle RBF
class RBFModel:
    def __init__(self, num_classes=10):
        self.num_classes = num_classes
        self.kmeans = KMeans(n_clusters=num_classes)
        self.log_reg = LogisticRegression(max_iter=1000)

    def fit(self, x, y):
        self.kmeans.fit(x)
        rbf_features = rbf_kernel(x, self.kmeans.cluster_centers_)
        self.log_reg.fit(rbf_features, y)

    def predict(self, x):
        rbf_features = rbf_kernel(x, self.kmeans.cluster_centers_)
        return self.log_reg.predict(rbf_features)

# Entraînement des modèles
model_maxout = create_maxout_model()
model_maxout.fit(x_train, y_train, epochs=5, verbose=0)

model_softmax = create_softmax_model()
model_softmax.fit(x_train, y_train, epochs=5, verbose=0)

model_rbf = RBFModel()
model_rbf.fit(x_train_flat, y_train)

# Génération des exemples adversariaux avec le modèle Maxout
x_adv = generate_adversarial_examples(model_maxout, x_test, y_test)

# Prédictions croisées
y_pred_maxout = np.argmax(model_maxout.predict(x_adv), axis=1)
y_pred_softmax = np.argmax(model_softmax.predict(x_adv), axis=1)
y_pred_rbf = model_rbf.predict(x_adv.reshape(-1, 28 * 28))

# Calcul des pourcentages
def calculate_statistics(y_true, y_pred_1, y_pred_2):
    both_wrong = (y_true != y_pred_1) & (y_true != y_pred_2)
    agree_when_wrong = both_wrong & (y_pred_1 == y_pred_2)
    return {
        "both_wrong": np.mean(both_wrong) * 100,
        "agree_when_wrong": np.mean(agree_when_wrong) * 100,
    }

# Résultats pour Maxout vs Softmax
stats_maxout_softmax = calculate_statistics(y_test, y_pred_maxout, y_pred_softmax)
# Résultats pour Maxout vs RBF
stats_maxout_rbf = calculate_statistics(y_test, y_pred_maxout, y_pred_rbf)

# Affichage des résultats
print("Maxout vs Softmax:")
print(f" - Les deux modèles se trompent : {stats_maxout_softmax['both_wrong']:.2f}%")
print(f" - Accord sur les erreurs : {stats_maxout_softmax['agree_when_wrong']:.2f}%")

print("Maxout vs RBF:")
print(f" - Les deux modèles se trompent : {stats_maxout_rbf['both_wrong']:.2f}%")
print(f" - Accord sur les erreurs : {stats_maxout_rbf['agree_when_wrong']:.2f}%")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Maxout vs Softmax:
 - Les deux modèles se trompent : 97.67%
 - Accord sur les erreurs : 55.37%
Maxout vs RBF:
 - Les deux modèles se trompent : 50.69%
 - Accord sur les erreurs : 20.64%


Ici, le réseau softmax (principalement linéaire) a une forte tendance à reproduire les prédictions du réseau Maxout, ce qui montre l'importance des composantes linéaires dans le processus de généralisation des modèles, tandis que le réseau RBF lui (moins linéaire) est bien plus éloigné du réseau Maxout.